In [1]:
import numpy as np
import pandas as pd
from collections import Counter
from nltk.corpus import stopwords
import re

In [2]:
data = pd.DataFrame()

In [3]:
with open('reviews.txt','r') as f:
    data['review'] = np.array(map(lambda x:x[:-1], f.readlines()))

with open('labels.txt','r') as f:
    data['label'] = np.array(map(lambda x:x[:-1].lower(), f.readlines()))

In [4]:
data.head()

,review,label
0,bromwell high is a cartoon comedy . it ran at ...,positive
1,story of a man who has unnatural feelings for ...,negative
2,homelessness or houselessness as george carli...,positive
3,airport starts as a brand new luxury pla...,negative
4,brilliant over acting by lesley ann warren . ...,positive


In [5]:
stoplist = stopwords.words("english")

In [6]:
def review(val):
    val = str(val)
    if not val:
        return ''
    regex = re.compile('([^\s\w]|_)+')
    review = regex.sub('', val).lower()
    review = review.split(" ")
    for word in list(review):
        if word in stoplist:
            review.remove(word)  
            
    review = " ".join(review)
    return review

def clean_dataframe(data):
#     data = data.dropna(how="any")    
    data['review'] = data['review'].map(review)
    return data

In [7]:
data = clean_dataframe(data)

In [8]:
data.head()

,review,label
0,bromwell high cartoon comedy ran time program...,positive
1,story man unnatural feelings pig starts openi...,negative
2,homelessness houselessness george carlin stat...,positive
3,airport starts brand new luxury plane lo...,negative
4,brilliant acting lesley ann warren best dram...,positive


In [9]:
words_set = set(' '.join(data['review']).split())
word_int = {word : index for index, word in enumerate(words_set)}

In [10]:
X = pd.DataFrame(np.zeros([data.shape[0], len(words_set)]), columns=range(len(words_set)))

In [11]:
y = (data['label'] == 'positive').astype(int)

In [12]:
for i in range(data.shape[0]):
    indexs = [word_int[word] for word in data.loc[i, 'review'].split()]
    X.loc[i, indexs] = 1

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
test_size = 0.3
index = int(X.shape[0] * test_size)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3 ,random_state = 20)

In [ ]:
from sklearn.svm import SVC
clf = SVC(C=1.0, kernel='rbf', random_state=10)
clf.fit(X.loc[:index],  y.loc[:index])
print clf.score(X.loc[index:], y.loc[index:])

In [15]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=10)
clf.fit(X.loc[:index],  y.loc[:index])
print clf.score(X.loc[index:], y.loc[index:])

0.844971428571


In [16]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=10, random_state=10)
clf.fit(X.loc[:index],  y.loc[:index])
print clf.score(X.loc[index:], y.loc[index:])

0.7324
